In [54]:
from bs4 import BeautifulSoup as bs
import lxml
import nltk
import os
import string

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import pandas as pd

df = pd.read_json("people.json")

import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np
np.random.seed(400)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jason\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jason\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
def extract_info(filepath,words):
    
    dictionary_of_interest = {}
    
    with open(filepath,"r",encoding="utf8") as file:
            
        content = file.readlines()
        content = "".join(content)
        
        bs_content = bs(content, "lxml")
        
        unique_id = bs_content.find("tei").attrs["xml:id"]
        
        dictionary_of_interest["unique_id"] = unique_id
        
        letter_details = bs_content.find_all("correspaction")
        
        for deets in letter_details:
            
            if deets.attrs["type"] == "sent":
                
                if "when" in list(deets.date.attrs.keys()):
                    dictionary_of_interest["date"] = deets.date.attrs["when"]
                
                try:
                    dictionary_of_interest["sender"] = deets.persname.text
                    if dictionary_of_interest["sender"] != "Darwin, C. R.":
                        file.close()
                        return "Not Darwin Writing"
                except AttributeError:
                    dictionary_of_interest["reciever"] = deets.orgname.text
                
                try:
                    dictionary_of_interest["sender_bio"] = deets.persname.attrs["key"]
                except AttributeError:
                    dictionary_of_interest["sender_bio"] = "None Available"
                except KeyError:
                    dictionary_of_interest["sender_bio"] = "None Available"
                    
            if deets.attrs["type"] == "received":
                
                try:
                    dictionary_of_interest["reciever"] = deets.persname.text
                except AttributeError:
                    dictionary_of_interest["reciever"] = deets.orgname.text
                    
                try:
                    dictionary_of_interest["reciever_bio"] = deets.persname.attrs["key"]
                except AttributeError:
                    dictionary_of_interest["reciever_bio"] = "None Available"
                except KeyError:
                    dictionary_of_interest["reciever_bio"] = "None Available"
                    
        try:
            free_text = bs_content.find_all("div",{"type":"transcription"})[0].p.text
        except AttributeError:
#             print(bs_content) 
            free_text = ""
            

        # cleaning of the data
        dictionary_of_interest["body"] = (free_text.lower()).translate(str.maketrans('','',string.punctuation))
        text_tokens = word_tokenize(dictionary_of_interest["body"])
#         no_stop_words = [word for word in text_tokens] # if not word in stopwords.words()
        for word in text_tokens:
            words.append(word)
#         print(no_stop_words)!

        file.close()
        
    return dictionary_of_interest

def generate_feature_data(free_text,feature_set):
    
    feature_bools = []
    
    for word in feature_set:
        feature_bools.append(1*(word in free_text))
        
    return feature_bools

In [12]:
path = "dcp-data/letters/"
files = os.listdir(path)
files = files[1:]
words = []
# print(files[1])
i = 0
cap = 0

if cap == 0:
    cap = len(files)
    
data_set = []
for file_target in files:
    dict_test = extract_info(path+file_target,words)
    if dict_test != "Not Darwin Writing":
        data_set.append(dict_test)
#     print(dict_test)
    if i == cap:
        break
    elif i < cap:
        i += 1
    else:
        print("Failed loop")
        break
    print(round((i/cap)*100,2),end="\r"*(i!=cap))

print("")
print("Analysis Finished")
print(f"When all words are extracted, we have got a dataset of {len(words)} words")

100.0
Analysis Finished
When all words are extracted, we have got a dataset of 705858 words


In [13]:

if "dump.txt" not in os.listdir("."):
    unique_words = {}
    for counter, word in enumerate(words):
        try:
            unique_words[word] += 1
        except KeyError:
            unique_words[word] = 1
        print(round(((counter+1)/len(words))*100,2),end="\r")

    sorted_unique_words = {key: value for key, value in sorted(unique_words.items(), key=lambda item: item[1],reverse=True)}

    print(list(sorted_unique_words.keys())[:1000])
    feature_words = list(sorted_unique_words.keys())[:4000]
    with open("dump.txt","w",encoding="utf8") as output:
        for word in feature_words:
            try:
                output.write(word +"\n")
            except:
                print(word)

    output.close()

with open('dump.txt', encoding="utf8") as f:
    lines = f.readlines()
    
lines_cleaned = []
for word in lines:
    word = word[:-1]
    lines_cleaned.append(word)
    
feature_words = lines_cleaned
# print(sum(list(sorted_unique_words.values())[:4000]))
# print(sum(list(sorted_unique_words.values())[4000:]))

In [42]:
test = generate_feature_data(data_set[0]["body"],feature_words)

complete_data = []
targets = []
for dictionary in data_set:
    free_text = dictionary["body"]
    try:
        reciever_id = dictionary["reciever_bio"]
    except:
        print(dictionary)
        continue
    number_key = reciever_id[21:-4]
    
    boolean_set = generate_feature_data(free_text,feature_words)
    try:
        gender = df[df["id"]=="DCP-IDENT-"+str(number_key)]["sex"].iloc[0]
    except:
        gender = "NotAvailable"
#         print(number_key)
#         print(reciever_id)
#         print(df[df["id"]=="DCP-IDENT-"+str(number_key)])
        
    complete_data.append(boolean_set)
    targets.append(gender)
#     if counter == 10:
#         break
#     else:
#         counter += 1
# print(dict_test["sender"])
# print(dict_test["reciever"])
# print(test)

123
../nameregs/nameregs_123.xml
Empty DataFrame
Columns: [id, sex, name, keywords]
Index: []
2944
../nameregs/nameregs_2944.xml
Empty DataFrame
Columns: [id, sex, name, keywords]
Index: []
2944
../nameregs/nameregs_2944.xml
Empty DataFrame
Columns: [id, sex, name, keywords]
Index: []
2637
../nameregs/nameregs_2637.xml
Empty DataFrame
Columns: [id, sex, name, keywords]
Index: []
4407
../nameregs/nameregs_4407.xml
Empty DataFrame
Columns: [id, sex, name, keywords]
Index: []

None Available
Empty DataFrame
Columns: [id, sex, name, keywords]
Index: []
4407
../nameregs/nameregs_4407.xml
Empty DataFrame
Columns: [id, sex, name, keywords]
Index: []
4407
../nameregs/nameregs_4407.xml
Empty DataFrame
Columns: [id, sex, name, keywords]
Index: []
5147
../nameregs/nameregs_5147.xml
Empty DataFrame
Columns: [id, sex, name, keywords]
Index: []
4110
../nameregs/nameregs_4110.xml
Empty DataFrame
Columns: [id, sex, name, keywords]
Index: []
7808
../nameregs/nameregs_7808.xml
Empty DataFrame
Columns: [

In [53]:
unique_tags = {}
for val in targets:
    try:
        unique_tags[val] += 1
    except:
        unique_tags[val] = 1
        
print(unique_tags)
print(list(unique_tags.values())[0]/sum(unique_tags.values()))

{'M': 7277, 'NotAvailable': 306, '': 225, 'F': 324}
0.8948598130841121


In [60]:
X_train, X_test, y_train, y_test = train_test_split(
    complete_data, targets, test_size=0.2, random_state=42
)

In [61]:
classifier = KNeighborsClassifier(30)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)
print(score)

0.8813767670559312


In [63]:
classifier = DecisionTreeClassifier(max_depth=10)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

KeyboardInterrupt: 

In [68]:
out = classifier.apply(X_test)
print(out)

[21 57 57 ... 57 57 57]
